In [216]:
import pandas as pd
import numpy as np

# Loading Dataset
To load the dataset, we can use the `read_csv()` function from the pandas library :

```
python
df_part1 = pd.read_csv('./Sleep_health_and_lifestyle_dataset.csv')
df_part2 = pd.read_csv('./Sleep_health_and_lifestyle_dataset_part_2.csv')
```
These two lines of code will load the dataset from the CSV files `Sleep_health_and_lifestyle_dataset.csv` and `Sleep_health_and_lifestyle_dataset_part_2.csv` respectively and store them in the variables `df_part1` and `df_part2`.


In [217]:
df_part1 = pd.read_csv('./Sleep_health_and_lifestyle_dataset.csv')
df_part2 = pd.read_csv('./Sleep_health_and_lifestyle_dataset_part_2.csv')

# Merging two parts of the dataset to create a single one

In [ ]:
df = pd.concat((df_part1, df_part2))
print(df.shape)
df.head()

## Exploring the data set

In [ ]:
df.info()

In [ ]:
df.describe()

# Dropping Irrelevant Feature Person ID

In [221]:
df.drop('Person ID', axis=1, inplace=True)

# Handling NA values  

In [ ]:
df.isnull().sum()

### NA values found in 'Sleep Disorder' indicates that the person doesn't have any sleep disorders, so instead of treating it as NA value we will assign 'None' to it 

In [223]:
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')

In [ ]:
df.isnull().sum()

In [ ]:
df

## Handling the 'Blood Pressure' column by splitting it into Low BP and High BP

In [226]:
lowRow = []
highRow = []


for val in df['Blood Pressure']:
    sp = val.split('/')

    highRow.append(int(sp[0]))
    lowRow.append(int(sp[1]))

df['LowBP'] = np.array(lowRow)
df['HighBP'] = np.array(highRow)


In [227]:
df = df.drop('Blood Pressure', axis = 1)

In [ ]:
df

## Using LabelEncoders to Encode the Categorical Data
#### We also store a dictionay of encoders so that we can also Transform the unseen data into the same format

In [229]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
catData = [key for key in df if (df[key].dtype == object)]
numData = [key for key in df if (df[key].dtype != object and key != 'Stress Level')]
print('categorical features are :',catData)

In [ ]:
label_encoding_dict = {}
for category in catData:
    labelEncoder = LabelEncoder()
    labelEncoder.fit(df[category])
    df[category] = labelEncoder.transform(df[category])
    label_encoding_dict[category] = labelEncoder
df

# Distribution of target features

In [ ]:
import matplotlib.pyplot as plt
count=df['Sleep Disorder'].value_counts()
plt.bar(label_encoding_dict['Sleep Disorder'].inverse_transform(count.keys()),count)
plt.xlabel('Sleep Disorder')
plt.ylabel('Number of people')
plt.show()

# Boxplot of Features

In [ ]:
import matplotlib.pyplot as plt

columns = ['Gender', 'Age', 'Occupation', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'BMI Category', 'Heart Rate', 'Daily Steps', 'LowBP', 'HighBP']

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10))

for i, column in enumerate(columns):
    ax = axes[i // 4, i % 4]
    ax.boxplot(df[column])
    ax.set_title(column)

plt.tight_layout()
plt.show()

## Correlation analysis

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Calculate the correlation matrix
corr_matrix = df.corr()

# Plot the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Features')
plt.show()

The correlation heatmap provides insights into the relationships between different features in the dataset

# Standardisation of Data

In [ ]:
numData

In [ ]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
for f in numData:
    df[f] = StandardScaler().fit_transform(df[[f]])
df

In [ ]:
label_encoding_dict['Sleep Disorder'].inverse_transform([0])

In [238]:
from sklearn.decomposition import PCA

In [239]:
from sklearn.preprocessing import PolynomialFeatures

# Regression
tgt1 = df['Stress Level'] 
feat1 = df.drop(columns = ['Stress Level','Sleep Disorder']) 


In [ ]:
feat1

In [241]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
scoreList = []

for i in range(1, 5):

    poly = PolynomialFeatures(i)
    x = poly.fit_transform(feat1)

    X_train, X_test, y_train, y_test = train_test_split(x, np.array(tgt1), random_state = 42)

    model = LinearRegression()
    model.fit(X_train,y_train)
    pred = model.predict(X_test)

    scoreList.append(r2_score(y_test, pred))

scoreList

In [ ]:
xList = [i for i in range(1, 5)]

plt.xlabel("Degree of polynomial feature")
plt.ylabel("R2 Score")
plt.title("Graph of Degree vs R2 Score")
plt.plot(xList, scoreList)